<a href="https://colab.research.google.com/github/AndrewRucker/Fantasy-Football-and-Betting/blob/main/BestBall/Underdog/Rookie-and-Sophomores/Rookie_and_Soph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Library imports and dataset creation

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import ipywidgets as widgets
from IPython.core.pylabtools import figsize
from IPython.display import display

sns.set_context(font_scale=2)

# to bypass warnings in various dataframe assignments
pd.options.mode.chained_assignment = None

In [ ]:
#Read in CSV to dataframe
url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/Underdog/Rookie-and-Sophomores/RookSophTeams.csv'
df = pd.read_csv(url, index_col='Team')
df = df.dropna(how='all')

df

In [ ]:
#Creation of dataset with player drafted with value according to ADP by draft date
some_values = ('Kenny Pickett', 'Matt Corral', 'Malik Willis', 'Sam Howell', 'Desmond Ridder', 'Trevor Lawrence', 'Justin Fields', 'Davis Mills', 'Mac Jones', 'Zach Wilson')
df_date = df.drop(columns=['POS', 'Round', 'Pick', 'ADP', 'Cumulative Val'])
df_date = df_date.loc[df_date['Name'].isin(some_values)]
df_date


#### Initial data manipulation

In [ ]:
df1 = df.groupby('Team')['POS'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

df1['RB'] = df1['POS'].str.count('RB')
df1['W/T'] = df1['POS'].str.count('W/T')
df1['QB'] = df1['POS'].str.count('QB')

df1['build'] = df1['QB'].astype(str) + "/" + df1['RB'].astype(str) + "/" + df1['W/T'].astype(str)
df1 = df1.drop(columns=['POS', 'RB', 'W/T', 'QB'])

In [ ]:
df2 = pd.merge(df, df1, how='left', on = 'Team')
df3 = df2.groupby('Team')['Name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df2 = pd.merge(df2, df3, how='left', on='Team')

In [ ]:
final_df = df2.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value'])
final_df = final_df.drop_duplicates()
final_df ['Players'] = final_df['Name_y']
final_df = final_df.drop(columns=['Name_y'])
final_df

split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)

In [ ]:
df1 = pd.merge(df1, df2, how='left', on = 'Team')
df1 = df1.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value', 'Name_y'])
df1

#### Player/Build viz and stats

In [ ]:
#Exposure calc
exp_df = df2['Name_x'].value_counts()
exp_df = exp_df.reset_index()
exp_df['Count'] = exp_df['Name_x']
exp_df['%'] = round((exp_df['Count'] / df2['Team'].max()) * 100, 1)
exp_df = exp_df.drop(columns=['Name_x'])
exp_df.rename({'index' : 'Player'}, axis='columns', inplace=True)
exp_df

In [ ]:
#Player Exposure
alt.Chart(exp_df).mark_bar().encode(
    x='Count',
    y=alt.Y("Player", sort='-x'),
    tooltip=['Player','Count', '%']
).properties(height=700)

In [ ]:
#My draft tendencies for QBs over time, by value

stripplot =  alt.Chart(df_date, width=40).mark_circle(size=8).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('Name'),
    tooltip=['Name', 'Draft Start', 'Value'],
    color=alt.Color('Value', scale=alt.Scale(scheme='orangered')),
    column=alt.Column(
        'Draft Start',
        header=alt.Header(
            labelAngle=-90,
            titleOrient='bottom',
            labelOrient='top',
            labelAlign='right',
            labelPadding=3,
        ),
    ),
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_range(
    category={'scheme' : 'dark2'}
)

stripplot

In [ ]:
#Team Build by draft date, with cumulative ADP value

alt.Chart(df1).mark_circle(size=40).encode(
    x='Draft Start',
    y='Cumulative Val',
    color='build_x',
    tooltip=['Draft Start','Team', 'build_x', 'Cumulative Val']
).interactive()

In [ ]:
#Breakdown of all builds
build_count = final_df['build'].value_counts()
build_count = build_count.reset_index()
build_count['Count'] = build_count['build']
build_count['%'] = round((build_count['Count'] / df2['Team'].max()) * 100, 1)
build_count = build_count.drop(columns=['build'])
build_count.rename({'index' : 'Build'}, axis='columns', inplace=True)
build_count

In [ ]:
#Cumulative value description
cum_val_sum = final_df['Cumulative Val'].describe()
cum_val_sum

#### Player team by round search

In [ ]:
#Function that returns all teams with player passed, along with the builds used with the player
def player_team_search(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 12
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])

      #Building out round by round breakdown for teams with given player
      split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
      split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)
      split_df = split_df.merge(with_player, on=['Team', 'Team'], how='right')
      split_df = split_df.drop_duplicates(subset=['Team'])
      print("Round by Round selections with " + ', '.join(players) + " on the team \n")
      while i <= rounds:
        round_count = split_df[i].value_counts().rename_axis('Player').reset_index(name='Times Selected')
        round_count = round_count.sort_index()
        print("===============================================================")
        print("                                    Round " + str(i) + " \n")
        chart = bars = alt.Chart(round_count).mark_bar().encode(
            x=alt.X('Times Selected', axis=alt.Axis(labels=True), title=""),
            y=alt.Y('Player', axis=alt.Axis(labels=True), sort='-x', title=""),
            tooltip=['Player', 'Times Selected'],
            color=alt.Color('Player', legend=None, scale=alt.Scale(scheme=alt.SchemeParams(name='orangered')))
        ).interactive()
        chart.display()
        i=i+1
        
      #List of teams with given player on roster
      #print("===========================================================")
      #print("Full Teams with " + ', '.join(players) + " on the roster \n")
      #print(with_player['Players'].value_counts())

#### Build Count with Given Players

In [ ]:
#Build Count with given players function
def builds_with_players(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 12
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])    
      print("Structural builds with " + ', '.join(players) + " on the roster")
      print(with_player['build'].value_counts())
      chart = (alt.Chart(with_player).mark_circle(size=30).encode(
      x='Draft Start',
      y='Cumulative Val',
      color='build',
      tooltip=['Draft Start','Team', 'build', 'Cumulative Val']
      ).interactive())
      chart.display()
      print("\n")

#### To use while drafting

In [ ]:
players_to_search = ['Najee Harris', 'Trevor Lawrence']
build = builds_with_players(players_to_search)
player_team_search(players_to_search)